# RAG-Chatbot 

In [8]:
import pandas as pd
import os
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from neo4j import Query, GraphDatabase, RoutingControl, Result
from dotenv import load_dotenv
import gradio as gr
import time
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

## Get Credentials

In [4]:
env_file = 'credentials.env'

In [9]:
if os.path.exists(env_file):
    load_dotenv(env_file, override=True)

    # Neo4j
    HOST = os.getenv('NEO4J_URI')
    USERNAME = os.getenv('NEO4J_USERNAME')
    PASSWORD = os.getenv('NEO4J_PASSWORD')
    DATABASE = os.getenv('NEO4J_DATABASE')

    # AI
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
    LLM = os.getenv('LLM')
    EMBEDDINGS_MODEL = os.getenv('EMBEDDINGS_MODEL')
else:
    print(f"File {env_file} not found.")

## Setup Connection to Database

Setup connection to the database with the Python Driver

In [12]:
driver = GraphDatabase.driver(
    HOST,
    auth=(USERNAME, PASSWORD)
)

In [13]:
driver.execute_query(
    """
    MATCH (n) RETURN COUNT(n) as Count
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

,Count
0,917


## Create RAG-application

For the the chatbot we both need an Embedding-model and LLM. Create both below:

In [16]:
embedding_model = OpenAIEmbeddings(
    model=EMBEDDINGS_MODEL,
    openai_api_key=OPENAI_API_KEY
)

In [17]:
llm = ChatOpenAI(temperature=0, model=LLM)
llm.model_name

'gpt-4o'

### Retrieval Queries

To illustrate the difference between a "Regular" Vector Search and GraphRAG we create different retrieval queries.

In [40]:
def get_context_vector_search(search_prompt):
    query_vector = embedding_model.embed_query(search_prompt)
    
    similarity_query = """ 
        CALL db.index.vector.queryNodes("chunk-embeddings", 5, $query_vector) YIELD node, score
        WITH node as chunk, score ORDER BY score DESC
        MATCH (d:Document)<-[:PART_OF]-(chunk)
        RETURN score, d.file_name as file_name, chunk.page as page, chunk.chunk AS chunk
       """
    results = driver.execute_query(
        similarity_query,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        query_vector=query_vector,
        result_transformer_= lambda r: r.to_df()
    )
    print(results)
#    context = "Related documents: \n\n" + "\n\n".join(["file_name: " + record['file_name'] + "\n" + "page: " + str(record['page'] + 1) + "\n" + "text: " + record['chunk'] + "\n" for record in results.records])
    return results

In [41]:
# def get_context_graphrag(search_prompt):
    
#     return context, definitions

Function to retrieve the client name from a client id

### Prompts 

Prompt for vector search (without definitions)

In [42]:
def generate_prompt_vector_search(search_prompt, context):
    prompt_template = """

    You are a chatbot on Rabobank product. Your goal is to help people with questions on product policies.  
    A user will come to you with questions on their policy. Their questions must be answered based on the relevant documents of the policy.
    
    The question is the following: 
    {search_prompt}
    Always respond in the language in which the question was asked. So, do not respond in a different language.
    
    The context is the following: 
    {context}
    
    """
    prompt = PromptTemplate.from_template(prompt_template)
    
    theprompt = prompt.format_prompt(search_prompt=search_prompt, context=context)
    return theprompt

Prompt for GraphRAG

## Some examples to test the models

For every example there can be chosen between GraphRAG and vector search. 

In [44]:
search_prompt = 'Wat wordt bedoelt met de Rabofoon?'

context = get_context_vector_search(search_prompt)
theprompt = generate_prompt_vector_search(search_prompt, context)
llm(theprompt.to_messages()).pretty_print()

      score                                        file_name  page  \
0  0.565576  Payment and Online Services Terms Sept 2022.pdf    33   
1  0.565462  Payment and Online Services Terms Sept 2022.pdf    58   
2  0.563671  Payment and Online Services Terms Sept 2022.pdf    32   
3  0.563387  Payment and Online Services Terms Sept 2022.pdf    49   
4  0.563151  Payment and Online Services Terms Sept 2022.pdf    36   

                                               chunk  
0  te geven, aangifte te doen bij de politie of i...  
1  biljetten overdragen aan de autoriteiten. Ook ...  
2  Wij delen u de resultaten mee. De kosten hierv...  
3  Hoofdstuk 6 Betalen en ontvangen Voorwaarden b...  
4  Hoofdstuk 5 Betaalopdrachten37 Voorwaarden bet...  
================================== Ai Message ==================================

De term "Rabofoon" verwijst naar een telefonische dienst van Rabobank waarmee klanten hun bankzaken kunnen regelen via de telefoon. Het is een manier om toegang te kri

## Gradio Chatbot that uses RAG and GraphRAG

Example code is coming from Gradio documentation: [Creating a custom chatbot with blocks](https://www.gradio.app/guides/creating-a-custom-chatbot-with-blocks#add-streaming-to-your-chatbot)

In [46]:
def user(user_message, history):
    return "", history + [[user_message, None]]

def get_answer(search_prompt, rag_method):
    if rag_method == "Vector-Search":
        context = get_context_vector_search(search_prompt)
        theprompt = generate_prompt_vector_search(search_prompt, context)
    else: 
    # rag_method == "GraphRAG"
        context, definitions = get_context_graphrag(search_prompt)
        theprompt = generate_prompt_graphrag(search_prompt, context, definitions)
    messages = llm(theprompt.to_messages())
    return messages.content

def bot(history, rag_method):
    bot_message = get_answer(history[-1][0], rag_method)
    history[-1][1] = ""
    for character in bot_message:
        history[-1][1] += character
        time.sleep(0.01)
        yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        label="Chatbot with RAG", 
        avatar_images=["https://png.pngtree.com/png-vector/20220525/ourmid/pngtree-concept-of-facial-animal-avatar-chatbot-dog-chat-machine-illustration-vector-png-image_46652864.jpg","https://d-cb.jc-cdn.com/sites/crackberry.com/files/styles/larger/public/article_images/2023/08/openai-logo.jpg"]
    )
    msg = gr.Textbox(label="Message")
    rag_method = gr.Radio(["Vector-Search", "GraphRAG"], label="RAG-method:")
    clear = gr.Button("Clear")


    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, [chatbot, rag_method], chatbot
    )
    
    clear.click(lambda: None, None, chatbot, queue=False)

    
demo.queue()
demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


      score                                        file_name  page  \
0  0.565131  Payment and Online Services Terms Sept 2022.pdf    33   
1  0.561806  Payment and Online Services Terms Sept 2022.pdf    50   
2  0.561150  Payment and Online Services Terms Sept 2022.pdf    50   
3  0.560151  Payment and Online Services Terms Sept 2022.pdf    32   
4  0.560105  Payment and Online Services Terms Sept 2022.pdf    32   

                                               chunk  
0  te geven, aangifte te doen bij de politie of i...  
1  138.  Betalen door uw betaalpas- of creditcard...  
2  139. Hoe betaalt u met een overschrijvingsform...  
3  Wij delen u de resultaten mee. De kosten hierv...  
4  Neem dan eerst contact met ons op. Ga naar rab...  


If you want to have the light-mode for the chatbot paste the following after the URL: /?__theme=light